# Control Ableton with OSC

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import os

# Set the working directory to the main directory
os.chdir(os.path.dirname(os.path.abspath('')))

# Verify the current working directory
print(os.getcwd())

/home/stef/projects/ai-amg


In [12]:
from chord_generator import ChordProgressionGenerator
from music_gen.ableton_controllers import AbletonOSCController, AbletonMetaController
# from chord_generator import ChordProgressionGenerator
from music_gen.helpers import chord_to_midi, transpose_midi_chords

In [ ]:
ctrl = AbletonMetaController()

In [13]:
chord_prog = ChordProgressionGenerator()
controller = AbletonOSCController()

prog = chord_prog.generate_progression(0.5, 0.5)
midi_chords, midi_bass = chord_to_midi(prog)
controller.clip.add_notes(0, 0, midi_chords)
# # controller.song.set_tempo(100) 
# arousal = -0.8 # between -1 and 1
# # Map arousal to arp rate (1-127 range)
# arp_rate = ((arousal + 1) / 2) * (127 - 1) + 1  # Rescale to 1-12
# # track number 2, first device (that is the instrument)
# controller.device.set_parameter(0, 0, 0, 1) # controlling frequency of filter